In [6]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
# NLTK VADER for sentiment analysis
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Jack\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
site_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}
tickers = ['MSFT', 'IBM', 'AMZN']
for ticker in tickers:
    url = site_url + ticker
    req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'})
    response = urlopen(req)
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response, features="html.parser")
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table

# Read one single day of headlines for 'AMZN'
ibm = news_tables['IBM']
# Get all the table rows tagged in HTML with <tr> into 'amzn_tr'
ibm_tr = ibm.findAll('tr')

In [3]:
for i, table_row in enumerate(ibm_tr):
    # Read the text of the element 'a'
    a = table_row.a.text
    # Read the text of the element 'td' 
    td = table_row.td.text
    # Print the contents of 'link_text' and 'data_text'
    print(a)
    print(td)
    # Exit after printing 4 rows of data
    if i == 3:
        break

IBM Knocks on the Door of Its 52-Week High, But Nobody Answers

            Today 10:45AM
        
S&P 500 Gains and Losses Today: Index Falls as Fed Signals Another Rate Hike This Year

            Sep-20-23 05:17PM
        
Arm, Instacart, IBM: Trending Tickers

            04:26PM
        
Why IBM Shares Were Rising Today

            02:40PM
        


In [4]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        text = x.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text])
        
parsed_news

[['MSFT',
  'Today',
  '12:45PM',
  '\n\r\n            Today 12:45PM\r\n        \n\n\n\nUS STOCKS-Wall St rebounds as Treasury yields retreat from 16-year highs; Ford rises\n\n\n(Reuters)'],
 ['MSFT',
  'Today',
  '12:17PM',
  '\n\r\n            12:17PM\r\n        \n\n\n\nU.K. regulator signals approval for Microsoft to buy Activision Blizzard\n\n\n(LA Times)'],
 ['MSFT',
  'Today',
  '11:23AM',
  '\n\r\n            11:23AM\r\n        \n\n\n\nMicrosoft/Activision: messy process yields a better outcome for consumers\n\n\n(Financial Times)'],
 ['MSFT',
  'Today',
  '11:12AM',
  '\n\r\n            11:12AM\r\n        \n\n\n\nSNAP, Microsoft Advertising to Power Sponsored Links in My AI\n\n\n(Zacks)'],
 ['MSFT',
  'Today',
  '10:59AM',
  "\n\r\n            10:59AM\r\n        \n\n\n\nMicrosoft's mobile keyboard app SwiftKey gains new AI-powered features\n\n\n(TechCrunch)"],
 ['MSFT', 'Today', '10:42AM', '\n10:42AM\nLoading…\n'],
 ['MSFT',
  'Today',
  '10:42AM',
  '\n\r\n            10:42AM\

In [5]:
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Set column names
c = ['ticker', 'date', 'time', 'headline']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_and_scored_news = pd.DataFrame(parsed_news, columns=c)

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "pandas\_libs\tslib.pyx", line 605, in pandas._libs.tslib.array_to_datetime
  File "pandas\_libs\tslibs\parsing.pyx", line 318, in pandas._libs.tslibs.parsing.parse_datetime_string
  File "C:\Users\Jack\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\dateutil\parser\_parser.py", line 1368, in parse
    return DEFAULTPARSER.parse(timestr, **kwargs)
  File "C:\Users\Jack\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\dateutil\parser\_parser.py", line 643, in parse
    raise ParserError("Unknown string format: %s", timestr)
dateutil.parser._parser.ParserError: Unknown string format: Today

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "pandas\_libs\tslib.pyx", line 616, in pandas._libs.tslib.array_to_datetime
TypeError: invalid strin